In [5]:
import base64
#import imageio
import IPython
import matplotlib.pyplot as plt
import os
# import reverb
import tempfile
#import PIL.Image

In [2]:
import sys
import os
sys.modules[__name__]
sys.path.append("/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src")
sys.path.append("/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages")
sys.path.append("/home/nils/anaconda3/bin/python")
print(os.getcwd())
print(sys.executable)
print(sys.path)

/tf
/usr/bin/python3
['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src', '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages', '/home/nils/anaconda3/bin/python']


In [7]:
import tensorflow as tf
from tf_agents.environments import suite_gym
from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils
import rospy
import tempfile

import moveo_training.scripts.moveo_inverse_kinematic

tempdir = tempfile.gettempdir()

ModuleNotFoundError: No module named 'tf_agents'

In [14]:
rospy.init_node("train_moveo_her")
env_name = 'MoveoIK-v0' # @param {type:"string"}


In [15]:
# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 1 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

In [16]:

import gym
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
    if env_name in env:
        print("Remove {} from registry".format(env))
        del gym.envs.registration.registry.env_specs[env]



Remove MoveoIK-v0 from registry


In [17]:
from gym.envs.registration import register
register(
        id='MoveoIK-v0',
        entry_point='moveo_training.scripts.moveo_inverse_kinematic:MoveoIKEnv',
        max_episode_steps=100,
    )


In [18]:
env = suite_gym.load(env_name)
env.reset()

###### Setting Group Moveit with 30 seconds wait...


[ERROR] [1628592267.434938, 0.000000]: NOT Initialising Simulation Physics Parameters
[WARN] [1628592267.439516, 0.000000]: Start Init ControllersConnection
[WARN] [1628592267.441013, 0.000000]: END Init ControllersConnection
/home/nils/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Trying to set traj


TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.56698257], dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)})

In [19]:
collect_env = suite_gym.load(env_name)
eval_env = suite_gym.load(env_name)

###### Setting Group Moveit with 30 seconds wait...


[ERROR] [1628592268.258598, 0.000000]: NOT Initialising Simulation Physics Parameters
[WARN] [1628592268.266199, 0.000000]: Start Init ControllersConnection
[WARN] [1628592268.267496, 0.000000]: END Init ControllersConnection


###### Setting Group Moveit with 30 seconds wait...


[ERROR] [1628592268.667965, 0.000000]: NOT Initialising Simulation Physics Parameters
[WARN] [1628592268.676959, 0.000000]: Start Init ControllersConnection
[WARN] [1628592268.678343, 0.000000]: END Init ControllersConnection


In [20]:
use_gpu = True

In [21]:
strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2021-08-10 12:44:28.803489: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-10 12:44:28.837823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-10 12:44:28.838215: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 970 computeCapability: 5.2
coreClock: 1.2155GHz coreCount: 13 deviceMemorySize: 3.94GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-10 12:44:28.838474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib:/opt/ros/noetic/lib:/opt/ros

In [22]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

In [23]:
with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

In [24]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

In [25]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

In [26]:
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)

In [27]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

[reverb/cc/platform/tfrecord_checkpointer.cc:150]  Initializing TFRecordCheckpointer in /tmp/tmp1q2zqp0j.
[reverb/cc/platform/tfrecord_checkpointer.cc:378] Loading latest checkpoint from /tmp/tmp1q2zqp0j
[reverb/cc/platform/default/server.cc:54] Started replay server on port 15400


In [28]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)


In [29]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

In [30]:

tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)



In [31]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)


In [32]:


random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())


In [33]:


rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)



In [37]:

initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()


Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj
Trying to set traj


[ WARN] [1628592485.926166921, 220.615000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame Gripper_Idol_Gear at time 220.606000 according to authority unknown_publisher
[ WARN] [1628592485.926213294, 220.615000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame Gripper_Servo_Gear at time 220.606000 according to authority unknown_publisher
[ WARN] [1628592485.926233343, 220.615000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame Link_1 at time 220.606000 according to authority unknown_publisher
[ WARN] [1628592485.926254812, 220.615000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame Link_2 at time 220.606000 according to authority unknown_publisher
[ WARN] [1628592485.926542428, 220.615000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame Link_3 at time 220.606000 according to authority unknown_publisher
[ WARN] [1628592485.926576142, 220.615000000]: TF_REPEATED_DATA ignoring data

In [ ]:


env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])


In [ ]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)


In [ ]:


saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)


In [ ]:

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]


In [ ]:

agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers)



In [ ]:

def get_eval_metrics():
  print("Starte evaluierung")
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  print("Evaluierung beendete")
  return results

metrics = get_eval_metrics()


In [ ]:


def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)


In [ ]:


# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  print("Hallo ich bin noch am trainineren")
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)
  
  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))
